<a href="https://colab.research.google.com/github/my612/Quantamania/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from itertools import product

# Define the distance matrix (example: 3 cities)
distance_matrix = np.array([
    [0, 1, 2],  # Distances from city 0
    [1, 0, 3],  # Distances from city 1
    [2, 3, 0]   # Distances from city 2
])

# Number of cities
n = distance_matrix.shape[0]

# QUBO matrix initialization
qubo_matrix = np.zeros((n**2, n**2))

# Penalty weights
P1 = 10  # Penalty for visiting a city more than once
P2 = 10  # Penalty for assigning multiple cities to one position

# Encode the objective function into QUBO
for i, j, t in product(range(n), range(n), range(n - 1)):
    if i != j:  # Avoid diagonal terms since d_ii = 0
        index_1 = i * n + t
        index_2 = j * n + (t + 1)
        qubo_matrix[index_1, index_2] += distance_matrix[i, j]

# Encode the constraints into QUBO
# Constraint 1: Each city is visited exactly once
for i in range(n):
    for t in range(n):
        index = i * n + t
        qubo_matrix[index, index] += P1 * (1 - 2 * np.sum([1 if t2 != t else 0 for t2 in range(n)]))
        for t2 in range(t + 1, n):
            index_2 = i * n + t2
            qubo_matrix[index, index_2] += 2 * P1

# Constraint 2: Each position in the route is occupied by exactly one city
for t in range(n):
    for i in range(n):
        index = i * n + t
        qubo_matrix[index, index] += P2 * (1 - 2 * np.sum([1 if i2 != i else 0 for i2 in range(n)]))
        for i2 in range(i + 1, n):
            index_2 = i2 * n + t
            qubo_matrix[index, index_2] += 2 * P2

# Print the QUBO matrix
print("QUBO Matrix:")
print(qubo_matrix)

# Save QUBO matrix to file (optional)
np.savetxt("qubo_matrix.txt", qubo_matrix, fmt="%.2f")


QUBO Matrix:
[[-60.  20.  20.  20.   1.   0.  20.   2.   0.]
 [  0. -60.  20.   0.  20.   1.   0.  20.   2.]
 [  0.   0. -60.   0.   0.  20.   0.   0.  20.]
 [  0.   1.   0. -60.  20.  20.  20.   3.   0.]
 [  0.   0.   1.   0. -60.  20.   0.  20.   3.]
 [  0.   0.   0.   0.   0. -60.   0.   0.  20.]
 [  0.   2.   0.   0.   3.   0. -60.  20.  20.]
 [  0.   0.   2.   0.   0.   3.   0. -60.  20.]
 [  0.   0.   0.   0.   0.   0.   0.   0. -60.]]
